In [1]:
import numpy as np
import pandas as pd

In [2]:
chinook_customers = pd.read_csv('C:\\Users\\Fabulous\\Downloads\\result\\Customer.csv')
chinook_genre = pd.read_csv('C:\\Users\\Fabulous\\Downloads\\result\\Genre.csv')
chinook_invoice = pd.read_csv('C:\\Users\\Fabulous\\Downloads\\result\\Invoice.csv')
chinook_invoice_line = pd.read_csv('C:\\Users\\Fabulous\\Downloads\\result\\InvoiceLine.csv')
chinook_track = pd.read_csv('C:\\Users\\Fabulous\\Downloads\\result\\Track.csv')

In [3]:
chinook_customers.head(2)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,NaN,Theodor-Heuss-Straße 34,Stuttgart,NaN,Germany,70174,+49 0711 2842222,NaN,leonekohler@surfeu.de,5


In [4]:
chinook_genre.head(2)

,GenreId,Name
0,1,Rock
1,2,Jazz


In [5]:
chinook_invoice.head(2)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,NaN,Theodor-Heuss-Straße 34,Stuttgart,NaN,Germany,70174,1.98
1,2,4,NaN,Ullevålsveien 14,Oslo,NaN,Norway,0171,3.96


In [6]:
chinook_invoice_line.head(2)

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1


In [7]:
chinook_track.head(2)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,NaN,342562,5510424,0.99


Which city has the best customers?

We would like to throw a promotional Music Festival in the city we made the most money. Write a query that returns the 1 city that has the highest sum of invoice totals. Return both the city name and the sum of all invoice totals.

In [8]:
best_customers_city = chinook_invoice.groupby(['BillingCity'])['Total'].sum()
best_customers_city = best_customers_city.nlargest(1).reset_index()
best_customers_city

,BillingCity,Total
0,Prague,90.24


Which countries have the most Invoices?

Use the Invoice table to determine the countries that have the most invoices. Provide a table of BillingCountry and Invoices ordered by the number of invoices for each country. The country with the most invoices should appear first.

In [9]:
highest_invoice_country = chinook_invoice.groupby(['BillingCountry'])['InvoiceId'].count().reset_index()
highest_invoice_country.sort_values(by = ['InvoiceId'], ascending = False, inplace = True)
highest_invoice_country.reset_index(drop = True, inplace = True)
highest_invoice_country.rename(columns = {'BillingCountry':'Billing_Country','InvoiceId':'Count of Invoice'}, 
                               inplace = True)
highest_invoice_country.head()

,Billing_Country,Count of Invoice
0,USA,91
1,Canada,56
2,Brazil,35
3,France,35
4,Germany,28


Who is the best customer?

The customer who has spent the most money will be declared the best customer. Build a query that returns the person who has spent the most money

In [10]:
best_customer = chinook_customers.merge(chinook_invoice, on = 'CustomerId', how='inner')
best_customer['Full_Name'] = best_customer[['FirstName', 'LastName']].agg(' '.join, axis=1)
best_customer = best_customer.groupby(['CustomerId','Full_Name'])['Total'].sum()
best_customer = best_customer.nlargest(1).reset_index()
best_customer

,CustomerId,Full_Name,Total
0,6,Helena Holý,49.62


Use your query to return the email, first name, last name, and Genre of all Rock Music listeners. Return your list ordered alphabetically by email address starting with A.

In [11]:
rock_customer_raw = chinook_customers.merge(chinook_invoice, on = 'CustomerId', how='inner')
rock_customer_raw = rock_customer_raw.merge(chinook_invoice_line, on = 'InvoiceId', how='inner')
rock_customer_raw = rock_customer_raw.merge(chinook_track, on = 'TrackId', how='inner')
rock_customer_raw = rock_customer_raw.merge(chinook_genre, on = 'GenreId', how='inner')
pd.set_option('display.max_columns', None)
rock_customer_raw.head(1)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId,InvoiceId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,InvoiceLineId,TrackId,UnitPrice_x,Quantity,Name_x,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice_y,Name_y
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3,98,NaN,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,3.98,531,3247,1.99,1,Experiment In Terra,253,3,20,NaN,2923548,547982556,1.99,Sci Fi & Fantasy


In [12]:
rock_customer_new = rock_customer_raw[['Email','FirstName','LastName','Name_y']]
rock_customer_new = rock_customer_new[rock_customer_new['Name_y']=='Rock']
rock_customer = rock_customer_new.copy()
rock_customer.sort_values(by = ['Email'], ascending = True, inplace = True)
rock_customer = rock_customer.drop_duplicates()
rock_customer.reset_index(drop=True, inplace=True)
rock_customer.rename(columns={'FirstName':'First_Name','LastName':'Last_Name','Name_y':'Genre'}, 
                     inplace=True)
rock_customer.head()

,Email,First_Name,Last_Name,Genre
0,aaronmitchell@yahoo.ca,Aaron,Mitchell,Rock
1,alero@uol.com.br,Alexandre,Rocha,Rock
2,astrid.gruber@apple.at,Astrid,Gruber,Rock
3,bjorn.hansen@yahoo.no,Bjørn,Hansen,Rock
4,camille.bernard@yahoo.fr,Camille,Bernard,Rock


We want to find out the most popular music Genre for each country. We determine the most popular genre as the genre with the highest amount of purchases. Write a query that returns each country along with the top Genre. For countries where the maximum number of purchases is shared return all Genres.

In [13]:
popular_genre = rock_customer_raw.groupby(['Country','Name_y']).agg({'InvoiceId':['count']})
popular_genre.columns = ['Number of Purchases']
popular_genre.reset_index(inplace=True)
popular_genre.head()

,Country,Name_y,Number of Purchases
0,Argentina,Alternative & Punk,9
1,Argentina,Easy Listening,2
2,Argentina,Jazz,2
3,Argentina,Latin,8
4,Argentina,Metal,7


In [14]:
popular_genre_1 = popular_genre.groupby(['Country'])['Number of Purchases'].transform(
    'max') == popular_genre['Number of Purchases']
popular_genre_country = popular_genre[popular_genre_1].reset_index(drop=True)
popular_genre_country.sort_values(by=['Number of Purchases'], ascending = False, inplace = True)
popular_genre_country.reset_index(drop=True, inplace=True)
popular_genre_country

,Country,Name_y,Number of Purchases
0,USA,Rock,157
1,Canada,Rock,107
2,Brazil,Rock,81
3,France,Rock,65
4,Germany,Rock,62
5,United Kingdom,Rock,37
6,Portugal,Rock,31
7,Czech Republic,Rock,25
8,India,Rock,25
9,Australia,Rock,22
